In [1]:
from trainer import PCOS_trainer
import torch
import os
import datetime 
import optuna
from optuna.samplers import GridSampler

Seed Fix: 42


In [2]:


def objective(trial):
    """
    Optuna가 반복적으로 호출하는 objective 함수.
    - trial: Optuna 내부에서 현재 시도하는 하이퍼파라미터를 담고 있는 객체
    """

    # 하이퍼파라미터 최적화
    label_smoothing = trial.suggest_uniform('label_smoothing', 0.0, 0.3)
    lr = trial.suggest_loguniform('lr', 1e-8, 1e-3)
    # label_smoothing = 0.19215890194834306
    # lr = 0.00018223330318900866
    
    patience = 18
    # scheduler 관련 unified parameters (모든 스케줄러에 동일하게 적용)
    scheduler_type = 'cosine'            # 스케줄러 종류: 'cosine' / 'plateau' / 'step' / 'multistep' / 'cyclic' / 'warmrestarts'
    scheduler_factor = 0.1               # ReduceLROnPlateau에서 사용 (학습률 감소 비율)
    scheduler_patience = patience // 2   # ReduceLROnPlateau에서 사용 (개선 없는 에폭 수)
    scheduler_T_max = 50 - 5             # CosineAnnealingLR: (epoch - warmup_epochs)
    scheduler_eta_min = 1e-6             # CosineAnnealingLR에서 사용 (최소 학습률)
    scheduler_step_size = 10             # StepLR에서 사용 (에폭 단위 감소 주기)
    scheduler_gamma = 0.5                # StepLR, MultiStepLR, CyclicLR 등에서 사용 (감소 비율)
    scheduler_milestones = [10, 30, 50]    # MultiStepLR에서 사용 (감소 시점)
    scheduler_max_lr_scale = 10.0        # CyclicLR에서 사용 (최대 lr 배수)
    scheduler_step_size_up = 5           # CyclicLR: 상승 단계 에폭 수
    scheduler_step_size_down = 5         # CyclicLR: 하강 단계 에폭 수
    scheduler_T0 = 10                    # CosineAnnealingWarmRestarts에서 사용 (첫 주기 길이)
    scheduler_T_mult = 1                 # CosineAnnealingWarmRestarts에서 사용 (이후 주기 배수)
    scheduler_eta_min_scale = 0.1        # CosineAnnealingWarmRestarts에서 사용 (최소 lr 비율)


    # Args 생성 및 초기화
    class Args:
        pass
    args = Args()
    
    #%% [WanDB]
    args.use_wandb = True
    args.project_name = "PCOS_Classification"
    args.experiment_name = "Model"

    #%% [Logging]
    args.exp_date = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    
    #%% [Args]
    args.device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
    args.datasheet_path = os.getenv('DATASHEET_PATH')
    args.data_dir = os.getenv('DATA_DIR')
    
    #%% [Model]
    args.model_name = 'convnext'
    args.model_version = 'l'  # 모델 버전 수정 (예: '18')
    
    #%% [Hyperparameters]
    args.loss_name = 'polyl1ce'  # None / 'polyl1ce' / 'poly1focal'
    args.epoch = 50              # 총 에폭 수
    args.warmup_epochs = 5        # Warmup 에폭 수
    args.patience = patience
    args.loss_label_smoothing = label_smoothing
    args.lr = lr


    #%% [Scheduler] Unified Parameters 설정
    args.scheduler_type = scheduler_type
    args.scheduler_factor = scheduler_factor
    args.scheduler_patience = scheduler_patience
    args.scheduler_T_max = scheduler_T_max
    args.scheduler_eta_min = scheduler_eta_min
    args.scheduler_step_size = scheduler_step_size
    args.scheduler_gamma = scheduler_gamma
    args.scheduler_milestones = scheduler_milestones
    args.scheduler_max_lr_scale = scheduler_max_lr_scale
    args.scheduler_step_size_up = scheduler_step_size_up
    args.scheduler_step_size_down = scheduler_step_size_down
    args.scheduler_T0 = scheduler_T0
    args.scheduler_T_mult = scheduler_T_mult
    args.scheduler_eta_min_scale = scheduler_eta_min_scale

    #%% [Data]
    # 여기서는 다중 클래스 실험으로 설정
    args.binary_use = True
    args.sampler_name = None  # 또는 'balanced' / 'weighted'
    args.use_kfold = True      # 단일 분할로 실험 (optuna에서는 빠른 실험을 위해)
    args.k_fold = 5
    
    #%% [Model]
    args.model_name = 'convnext'
    args.model_version = 'l'
    
    #%% [Experiment Name]
    args.experiment_name = f"KFold_convnext_l_params_search"
    
    # Trainer 생성 및 학습 실행
    trainer = PCOS_trainer(args)
    # best_val_auc = trainer.fit()  # 여기서는 k_fold_fit 대신 fit() 사용 (빠른 실험을 위해)
    best_val_auc = trainer.k_fold_fit()  #

    # Return 값: 최적의 검증 AUC (Optuna는 이 값을 최대화하려고 함)
    return best_val_auc

#### Optuna Hyperparameter Tuning : 수치형

In [3]:
def run_optuna_search():
    """
    #### Optuna Hyperparameter Tuning : 수치형
    Optuna Study를 생성/실행하고 최적 파라미터를 출력합니다.
    # 만약 무작위 탐색을 원한다면 아래 함수를 사용하세요.
    # 단, 이 경우 5번의 시도 중 일부 옵션이 중복 선택될 수 있으므로 모든 스케줄러 옵션이 반드시 테스트되지 않을 수 있습니다.
    """
    # direction='maximize' → best_val_auc가 높을수록 좋다고 판단합니다.
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20, show_progress_bar= True)  # n_trials=5: 총 5번 랜덤으로 시도 (실험 수 늘리면 더 많은 탐색 가능)

    print("\n===== [Optuna Search 결과] =====")
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best value (Val AUC): {study.best_trial.value:.4f}")
    print("Best hyperparams:")
    for k, v in study.best_trial.params.items():
        print(f" {k}: {v}")
run_optuna_search() #<-- Parameter Search 실행
        

[I 2025-03-05 00:24:28,346] A new study created in memory with name: no-name-8d21e30e-5bd8-45a0-a506-05eb66c94457


  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_1691899/2796427322.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  label_smoothing = trial.suggest_uniform('label_smoothing', 0.0, 0.3)
/tmp/ipykernel_1691899/2796427322.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-8, 1e-3)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dablro1232 (dablro12) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Large_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])
train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])
train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])
train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])
train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])
train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])


...

/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


train_preds: torch.Size([1887, 1]), train_labels: torch.Size([1887])


/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[W 2025-03-05 14:04:18,376] Trial 10 failed with parameters: {'label_smoothing': 0.021224462683491846, 'lr': 0.0009257179399119827} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_1691899/2796427322.py", line 95, in objective
    best_val_auc = trainer.k_fold_fit()  #
                   ^^^^^^^^^^^^^^^^^^^^
  File "/home/eiden/eiden/PCOS-roi-classification/v2/trainer.py", line 60, in k_fold_fit
    train_loss, train_metrics = self.trainer()
                                ^^^^^^^^^^^^^^
  File "/home/eiden/eiden/PCOS-roi-classification/v2/trainer.py", line 403, in trainer
    train_loss += loss.item()
                  ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-03-05 14:04:18,377] Trial 10 failed with value None.


KeyboardInterrupt: 

#### Grid Search Hyperparameter Tuning : 범주형

In [ ]:
# def run_optuna_grid_search():
#     # 그리드 서치할 하이퍼파라미터 공간 정의
#     search_space = {
#         'scheduler_type': ['cosine', 'StepLR', 'MultiStepLR', 'CyclicLR', 'CosineAnnealingWarmRestarts']
#     }

#     # GridSampler를 생성합니다.
#     sampler = GridSampler(search_space)

#     # Study 생성 (목표는 최대화)
#     study = sampler.create_study(direction='maximize')

#     # grid search에서는 n_trials를 하이퍼파라미터 조합 개수로 지정합니다.
#     study.optimize(objective, n_trials=len(search_space['scheduler_type']), show_progress_bar= True)

#     print("\n===== [Optuna Grid Search 결과] =====")
#     print(f"Best trial: {study.best_trial.number}")
#     print(f"Best value (Val AUC): {study.best_trial.value:.4f}")
#     print("Best hyperparams:")
#     for k, v in study.best_trial.params.items():
#         print(f"  {k}: {v}")
        
# run_optuna_grid_search()